In [31]:
# set up imports
import pandas as pd, numpy as np, seaborn as sns
from tabulate import tabulate
import statsmodels
import arch
import matplotlib
matplotlib.use('qt5agg')

# configure plot style
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams["figure.figsize"] = (9.5,4.15)
plt.rcParams['figure.constrained_layout.use'] = False
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['lines.linewidth'] = 0.8
save_plot_to =  r'C:\\Users\joche\OneDrive\03 TUM - TUM-BWL\Semester 8\01 Bachelorarbeit\04 Results\Plots/'

In [2]:
## OLD - THROW OUT ##

# load data from excel file
mydateparser = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
btc = pd.read_excel('Data/BTC_closing.xlsx',squeeze=True, parse_dates=[0], index_col=0, date_parser=mydateparser)

# crop data: 2015-08-07 to 2020-06-26
start_date = '2015-08-07'
end_date = '2020-06-26'
btc = btc[start_date : end_date]

In [4]:
# load data: new_btc.txt, created with M4
index_name = 'date'
btc = pd.read_csv('Data/btc_new.txt', index_col=0)
btc.index = pd.DatetimeIndex(btc.index, normalize=True).normalize()
btc

,btc
date,
2015-08-07,279.58
2015-08-10,264.47
2015-08-11,270.39
2015-08-12,266.38
2015-08-13,264.08
...,...
2020-06-18,9411.84
2020-06-19,9288.02
2020-06-22,9648.72


In [95]:
# define overview-printing function of dataframe
def dates_overview(dataframe):
    print('start_date:\t', dataframe.index[0])
    print('end_date:\t', dataframe.index[-1])
    print('len df:\t\t', len(dataframe))

# define train/test-splitting function of dataframe
def split_traintest_df(dataframe):
    train_size = int(len(dataframe) * 0.8)
    df_train, df_test = dataframe[0:train_size], dataframe[train_size:]
    # create dataframe for printout
    data = {'Dataframe': ['dataframe', 'df_train', 'df_test'],
            'date_start': [dataframe.index[0], df_train.index[0], df_test.index[0]],
            'date_end': [dataframe.index[-1], df_train.index[-1], df_test.index[-1]],
            'nobs': [len(dataframe), len(df_train), len(df_test)]}
    df_print = pd.DataFrame(data, columns=['Dataframe','date_start','date_end','nobs'])
    print(df_print)
    return df_train, df_test

# define log-taking and relabeling function
def log_of_df(dataframe):
    df_log = np.log(dataframe)
    new_cols = list()
    for i in df_log.columns:
        new_cols.append(i+'_log')
    df_log.columns = new_cols
    # fill na value of negative oil price on 2020-04-20 with 0
    df_log.fillna(value=0, inplace=True)
    return df_log

# define first difference-taking function of dataframe
def diff_of_df(dataframe):
    df_train_log_diff = dataframe.diff()
    # relabel columns
    new_cols = list()
    for i in df_train_log_diff.columns:
        new_cols.append(i+'_diff')
    df_train_log_diff.columns = new_cols
    return df_train_log_diff

# define return creating function of dataframe
def create_returns(dataframe_train):
    dataframe_train_log = log_of_df(dataframe_train)
    dataframe_returns = diff_of_df(dataframe_train_log)
    dataframe_returns.columns = ['btc_ret']
    return dataframe_train_log, dataframe_returns

# define log and ret + ACF & PACF plotting function for dataframe_log, dataframe_ret
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
def plot_acf_pacf(dataframe_log, dataframe_ret):
    fig, axs = plt.subplots(3,2)
    axs[0][0].plot(dataframe_log.iloc[1:], color='black')
    axs[0][0].set_title('btc_log')
    axs[0][1].plot(dataframe_ret.iloc[1:], color='black')
    axs[0][1].set_title('btc_ret')
    plot_acf(dataframe_log, color='black', ax=axs[1][0], markersize=2)
    plot_acf(dataframe_ret.iloc[1:], color='black', ax=axs[1][1], markersize=2)
    plot_pacf(dataframe_log, color='black', ax=axs[2][0], markersize=2)
    plot_pacf(dataframe_ret.iloc[1:], color='black', ax=axs[2][1], markersize=2)
    fig.tight_layout()
    return fig

# define adf- and pp-testing of dataframe with latex-printout on/off
from arch.unitroot import ADF, PhillipsPerron
def stationarity_tests(dataframe, latex):
    for col in dataframe:
        adf = ADF(dataframe[col])
        pp = PhillipsPerron(dataframe[col])
        if latex is False:
            # write summary as plain text to std.out
            print('Timeseries:\t',col,'\n',
                  adf.summary(),'\n\n',pp.summary(),'\n\n\n')
        else:
            # write summary as latex to file
            with open(save_plot_to + 'Stationarity_Tests_LaTeX.txt', 'a') as myfile:
                myfile.write('Timeseries:\t'+col+'\n'
                         +adf.summary().as_latex()+'\n\n'
                         +pp.summary().as_latex()+'\n\n\n')
                
# define stationaritiy table-generating function of a dataframe filepath
def stationarity_table_aslatex_from_df(filepath):
    # read in dataframe from .txt file
    stationarity_dataframe = pd.read_csv(save_plot_to+filepath, delimiter='\s+', header=0)
    # print dataframe as latex output
    print(tabulate(stationarity_dataframe, headers=stationarity_dataframe.columns, showindex=False, tablefmt="latex"))

# define auto-ARIMA function for a dataframe_ret
import pmdarima as pm
import warnings
warnings.filterwarnings('ignore')
def auto_arima_optimal_fit_of(dataframe_ret, max_p, max_q):
    model_auto = pm.auto_arima(dataframe_ret, start_p=0, start_q=0,
                               max_p=max_p, max_q=max_q,
                               test='adf',
                               m=1,
                               d=None,
                               trace=True,
                               error_action='ignore',
                               suppress_warnings=True,
                               seasonal=False,
                               stepwise=True)
    return model_auto

# define manually verifying function for dataframe_ret
import pmdarima as pm
import warnings
warnings.filterwarnings('ignore')
def arima_fit_of(dataframe_ret, max_p, max_q, latex):
    for p in range(max_p):
        for q in range(max_q):
            temp_model = pm.ARIMA(order=(p,0,q))
            temp_fitted = temp_model.fit(dataframe_ret)
            if latex is False:
                # write summary as plain text to std.out
                print('Current order: {0},0,{1}\n'.format(p,q),
                temp_fitted.summary(),'\n')
            else:
                # write summary as latex to file
                with open(save_plot_to + 'M2_ARIMA_configuratios_LaTeX.txt', 'a') as myfile:
                        myfile.write('Current order: {0},0,{1}\n'.format(p,q)+
                                     temp_fitted.summary().as_latex()+'\n\n')

# define rolling forecast function for dataframe and p,q values
from sklearn.metrics import mean_squared_error as mse
def arima_rolling_forecast(dataframe, p, q):
    # create train and test dataframes
    train_size = int(len(dataframe) * 0.8)
    dataframe_train, dataframe_test = dataframe[0:train_size], dataframe[train_size:]
    index = len(dataframe_train) - 1
    # initialize lists
    pred_val_btc = list()
    lo_conf_int_btc = list()
    up_conf_int_btc = list()
    ausreisser_ctr = 0
    # rolling forecast
    for i in range(len(dataframe_test)):
        # input data, dependent on i
        input_data = log_of_df(dataframe).diff().values[max(p,q)+1:index + i]
        # fit model and predict 1 step
        dataframe_arima = pm.ARIMA(order=(p,0,q))
        dataframe_arima_fitted = dataframe_arima.fit(y=input_data, disp=False)
        dataframe_arima_result = dataframe_arima_fitted.predict(n_periods=1, return_conf_int=True)
        # obtain absolute (inversed) btc values for mean prediction, upper- and lower confidence interval
        yhat_btc = np.exp(dataframe_arima_result[0][0] + np.log(dataframe).iloc[:,0][index+i])
        lo_conf_btc_val = np.exp(dataframe_arima_result[1][0][0] + np.log(dataframe).iloc[:,0][index+i])
        up_conf_btc_val = np.exp(dataframe_arima_result[1][0][1] + np.log(dataframe).iloc[:,0][index+i])
        if (dataframe.iloc[:,0][index+i+1] > up_conf_btc_val) or (dataframe.iloc[:,0][index+i+1] < lo_conf_btc_val):
            ausreisser_ctr += 1
        pred_val_btc.append(yhat_btc)
        lo_conf_int_btc.append(lo_conf_btc_val)
        up_conf_int_btc.append(up_conf_btc_val)
    # return [0]: mse, [1]: ausreisser ctr, [2]: predictions, [3]: lo_confint, [4]: up_confint
    return (np.sqrt(mse(pred_val_btc, dataframe_test.iloc[:,0].values)), ausreisser_ctr,
            pred_val_btc, lo_conf_int_btc, up_conf_int_btc)

# define series-generating and plotting function for dataframe_arima_pred_result
def plot_arima_pred_result(dataframe, dataframe_train, dataframe_test, dataframe_arima_pred_result):
    # make series for plotting pred. vs. actual
    index_pred = np.arange(len(dataframe_train) + 1, len(dataframe) + 1)
    pred_val_btc_series = pd.Series(btc_arima_pred_result[2], index=index_pred)
    test_series_btc = pd.Series(dataframe_test.iloc[:,0].values, index=index_pred)
    lo_conf_int_btc_series = pd.Series(btc_arima_pred_result[3], index=index_pred)
    up_conf_int_btc_series = pd.Series(btc_arima_pred_result[4], index=index_pred)
    # create plot: 
    fig, axs = plt.subplots(1,2)
    axs[0].plot(dataframe_train.iloc[:,0].values, label= r'$btc_T$', color='black')
    axs[0].plot(test_series_btc, label= r'$btc_{T+h}$', color='green')
    axs[0].plot(pred_val_btc_series, label= r'$\hat{btc}_{T+h}$', color= 'red')
    axs[0].fill_between(lo_conf_int_btc_series.index, lo_conf_int_btc_series, up_conf_int_btc_series, color='k', alpha=0.1)
    axs[0].legend(loc='upper left')
    axs[0].title.set_text('Gesamter Zeitraum')
    axs[1].plot(test_series_btc, label= r'$btc_{T+h}$', color='green')
    axs[1].plot(pred_val_btc_series, label= r'$\hat{btc}_{T+h}$', color= 'red')
    axs[1].fill_between(lo_conf_int_btc_series.index, lo_conf_int_btc_series, up_conf_int_btc_series, color='k', alpha=0.1)
    axs[1].legend(loc='upper left')
    axs[1].title.set_text('Vorhersage-Zeitraum')
    fig.suptitle('ARIMA(0,0,0) Einschrittprognose btc')
    return fig



In [ ]:
# overview for btc
dates_overview(btc)

In [13]:
# split btc in train and test dataset: proceed only with btc_train
btc_train, btc_test = split_traintest_df(btc)

   Dataframe date_start   date_end  nobs
0  dataframe 2015-08-07 2020-06-24  1112
1   df_train 2015-08-07 2019-07-01   889
2    df_test 2019-07-02 2020-06-24   223


In [36]:
# create log and returns for btc_train
btc_log, btc_ret = create_returns(btc_train)
btc_ret

,btc_ret
date,
2015-08-07,NaN
2015-08-10,-0.055561
2015-08-11,0.022138
2015-08-12,-0.014942
2015-08-13,-0.008672
...,...
2019-06-25,0.068426
2019-06-26,0.098867
2019-06-27,-0.151819


In [58]:
# plot btc_log and btc_ret with ACF and PACF, respectively
fig_1 = plot_acf_pacf(btc_log,btc_ret)

In [46]:
# save fig_1
fig_1.savefig(save_plot_to+'M2_fig_1.svg',format='svg')

In [51]:
# perform ADF and PP test on btc_log and btc_ret
stationarity_tests(btc_log,False)
stationarity_tests(btc_ret.iloc[1:],False)

Timeseries:	 btc_log 
    Augmented Dickey-Fuller Results   
Test Statistic                 -0.883
P-value                         0.794
Lags                                0
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary. 

      Phillips-Perron Test (Z-tau)    
Test Statistic                 -0.931
P-value                         0.778
Lags                               21
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary. 



Timeseries:	 btc_ret 
    Augmented Dickey-Fuller Results   
Test Statistic                -29.318
P-value                         0.000
Lags                                0
-------------------------------------

Trend: Constant
Cr

In [54]:
# generate latex output for M2_stationarity tests.txt
stationarity_table_aslatex_from_df('M2_stationarity tests.txt')

\begin{tabular}{lrrl}
\hline
 Variable   &     ADF &      PP & \ensuremath{<}-2.86(5\%)   \\
\hline
 btc\_log    &  -0.883 &  -0.931 & nein         \\
 btc\_ret    & -29.318 & -29.679 & ja           \\
\hline
\end{tabular}


In [ ]:
## ARCHIV: Genaue Testergebnisse für ADF und PP ##
# ADF- and PP- mit btc
from arch.unitroot import ADF, PhillipsPerron
adf = ADF(btc)
pp = PhillipsPerron(btc)
print(adf.summary().as_latex(),'\n\n\n', pp.summary().as_latex())

# ADF- and PP- mit btc_returns
from arch.unitroot import ADF, PhillipsPerron
adf = ADF(btc_returns[1:])
pp = PhillipsPerron(btc_returns[1:])
print(adf.summary().as_latex(),'\n\n\n', pp.summary().as_latex())

In [69]:
# obtain best auto arima model for btc_ret
btc_auto_arima = auto_arima_optimal_fit_of(btc_ret.iloc[1:],max_p=10,max_q=10)
btc_auto_arima.summary()

Performing stepwise search to minimize aic
Fit ARIMA(0,0,0)x(0,0,0,0) [intercept=True]; AIC=-2848.296, BIC=-2838.718, Time=0.132 seconds
Fit ARIMA(1,0,0)x(0,0,0,0) [intercept=True]; AIC=-2846.367, BIC=-2832.000, Time=0.062 seconds
Fit ARIMA(0,0,1)x(0,0,0,0) [intercept=True]; AIC=-2846.363, BIC=-2831.996, Time=0.439 seconds
Fit ARIMA(0,0,0)x(0,0,0,0) [intercept=False]; AIC=-2844.012, BIC=-2839.223, Time=0.080 seconds
Fit ARIMA(1,0,1)x(0,0,0,0) [intercept=True]; AIC=-2844.365, BIC=-2825.209, Time=0.499 seconds
Total fit time: 1.226 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  888
Model:                        SARIMAX   Log Likelihood                1426.148
Date:                Sat, 26 Sep 2020   AIC                          -2848.296
Time:                        12:53:43   BIC                          -2838.718
Sample:                             0   HQIC                         -2844.635
                                - 888                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0041      0.002      2.505      0.012       0.001       0.007
sigma2         0.0024    6.4e-05     36.818      0.000       0.002       0.002
===================================================================================
Ljung-Box (L1) (Q):                   0.07   Jarque-Bera (JB):               637.35
Prob(Q):                              0.79   Prob(JB):                         0.00
Heteroskedasticity (H):               1.62   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         7.14
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [65]:
# generate and print latex output for btc_auto_arima model
print(btc_auto_arima.summary().as_latex())
                

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}          &        y         & \textbf{  No. Observations:  } &    888      \\
\textbf{Model:}                  &     SARIMAX      & \textbf{  Log Likelihood     } &  1426.148   \\
\textbf{Date:}                   & Sat, 26 Sep 2020 & \textbf{  AIC                } & -2848.296   \\
\textbf{Time:}                   &     12:46:10     & \textbf{  BIC                } & -2838.718   \\
\textbf{Sample:}                 &        0         & \textbf{  HQIC               } & -2844.635   \\
\textbf{}                        &       - 888      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                   & \textbf{coef} & \textbf{std err} & \textbf{z} & \textbf{P$> |$z$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\midrule
\textbf{intercept} &       0.0041  &        0.002     &     2.505  &         0.012        &        0.001    &        0.007     \\
\textbf{sigma2}    &   

In [79]:
# manually verify ARIMA(p,0,q) variations for btc_ret and print them as output
arima_fit_of(btc_ret.iloc[1:],max_p=2,max_q=2,latex=False)

In [84]:
# rolling prediction with ARIMA(0,0,0) for btc
btc_arima_pred_result = arima_rolling_forecast(btc,0,0)
# printout mse and ausreisser
print('mse:\t\t', btc_arima_pred_result[0])
print('ausreisser:\t', btc_arima_pred_result[1])

(433.783168037022, 14, [10628.479661514059, 10845.971423288063, 12010.53709696853, 12337.921025273816, 12627.308187156434, 12208.491544042865, 11406.744281509793, 11865.053226924247, 10940.761269169267, 9516.463161658161, 9731.951977220395, 10708.68224241871, 10573.473264375265, 10384.89719624103, 9940.527275208462, 9850.809412059372, 9950.972392760483, 9909.3362514279, 9556.711450853809, 9644.902187522788, 10125.037437068037, 10440.818187168312, 10560.09738812417, 11852.805281856094, 11525.42761786824, 11990.713273822697, 12015.720735844932, 11911.79741146319, 11429.339401554791, 10940.008828732047, 10091.930609185918, 10351.866216140506, 10415.146740044629, 10959.013070925002, 10806.142701894642, 10178.270560551788, 10170.549797764463, 10448.487372748132, 10225.40645593642, 9792.367654389704, 9546.712210893278, 9634.716914497114, 10664.053279836915, 10636.00971693183, 10616.899582816803, 10393.736743014058, 10375.055637520654, 10155.15489701134, 10217.510225769995, 10450.187238762965

In [94]:
# plot arima(0,0,0) rolling forecast for btc_ret
fig_2 = plot_arima_pred_result(btc,btc_train,btc_test,btc_arima_pred_result)

In [96]:
# save fig_2
fig_2.savefig(save_plot_to+'M2_fig_2.svg',format='svg')

In [119]:
# plot diagnostics for ARIMA(0,0,0) for btc_ret and save as fig_3
arima_model = pm.ARIMA(order=(0,0,0))
arima_fitted = arima_model.fit(y=btc_ret.iloc[1:], disp=False)
arima_fitted.plot_diagnostics()
plt.tight_layout()
plt.savefig(save_plot_to+'M2_fig_3.svg',format='svg')

In [127]:
# test heteroskedast.
from statsmodels.tsa.arima_model import ARIMA
arima_model = ARIMA(endog=btc_ret.iloc[1:],order=(0,1,0))
arima_fitted = arima_model.fit()

In [ ]:
### btc_ret plotting, not relevant ###

In [27]:
# Einschrittprognose für btc_returns mit ARIMA(0,0,0)
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error as mse
import pmdarima as pm

history_ret = [x for x in btc_returns[1:]]
pred_val_ret = list()
lo_conf_int_ret = list()
up_conf_int_ret = list()

def einschritt_prognose_ret():
    aussreisser_ctr = 0
    for i in range(len(btc_returns_test)):
        model = pm.ARIMA(order=(0,0,0))
        model_fitted = model.fit(y=history_ret, disp=False)
        model_result = model_fitted.predict(n_periods=1, return_conf_int=True)
        yhat_ret = model_result[0][0]
        lo_conf_val_ret = model_result[1][0][0]
        up_conf_val_ret = model_result[1][0][1]
        
        if btc_returns[1427 + i] > up_conf_val_ret or btc_returns[1427 + i] < lo_conf_val_ret:
            aussreisser_ctr += 1
        pred_val_ret.append(yhat_ret)
        lo_conf_int_ret.append(lo_conf_val_ret)
        up_conf_int_ret.append(up_conf_val_ret)
        history_ret.append(btc_returns_test[i])

    # return mse_ret and ausreisser_ctr
    return (np.sqrt(mse(pred_val_ret, btc_returns_test.values)), aussreisser_ctr,
            pred_val_ret, lo_conf_int_ret, up_conf_int_ret)
    
pred_result_ret = einschritt_prognose_ret()
print(pred_result_ret)

In [29]:
# make series for plotting predicted vs. actual btc_returns
index_pred = np.arange(len(btc_returns_train) + 1, len(btc_returns) + 1)
pred_val_ret_series = pd.Series(pred_result_ret[2], index=index_pred)
test_series_ret = pd.Series(btc_returns_test.values, index=index_pred)
lo_conf_int_ret_series = pd.Series(pred_result_ret[3], index=index_pred)
up_conf_int_ret_series = pd.Series(pred_result_ret[4], index=index_pred)

# plot btc_returns einschrittprognosen für ARIMA(0,0,0)
fig_5 = plt.plot(btc_returns_train[1:].values, label= r'$X_T$')
plt.plot(test_series_ret, label= r'$X_{T+h}$', color='green')
plt.plot(pred_val_ret_series, label= r'$\hat{X}_{T+h}$', color= 'red')
plt.fill_between(lo_conf_int_ret_series.index, lo_conf_int_ret_series, up_conf_int_ret_series, color='k', alpha=0.1)
plt.legend(loc='upper left')
plt.title('Einschrittprognose btc_returns ARIMA(0,0,0)')

In [49]:
# Obtain ARIMA(0,0,0) residuals for btc_returns
model = pm.ARIMA(order=(0,0,0))
model_fitted = model.fit(y=btc_returns[1:], disp=False)
model_residuals = model_fitted.resid()

# Plot ARIMA(0,0,0) residual diagnostics for btc_returns
fig_6 = model_fitted.plot_diagnostics()

In [ ]:
# btc_returns ~ model_residuals
plt.plot(btc_returns[1:].values)
plt.plot(model_residuals)

In [9]:
# 
model_residuals.resid()

In [79]:
x = np.asarray(model_fitted.resid).squeeze()
x.ndim

In [58]:
# res expects attribute (not method!) resids: res.resids instead of res.resids()
# ndim(resids()) = 1, but ndim(resids = 0) -> Thats the problem
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
acorr_breusch_godfrey(res=model_fitted)

In [73]:
# no plan how to compare test-statistic
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(endog=btc_returns[1:], order=(0,0,0)) 
model_fit = model.fit()
model_fit.test_heteroskedasticity(method=None)

